<a href="https://colab.research.google.com/github/Romulomrs/MortesPalestina/blob/main/CleaningData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_json('dados.json')

In [55]:
pd.set_option('display.width', 1000)  # Aumenta o limite horizontal
pd.set_option('display.max_columns', None)  # Permite mostrar todas as colunas

In [56]:
# Preenche valores ausentes somente nas colunas numéricas
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(0)

In [57]:
df.fillna(0, inplace = True)
df = df.drop(['ext_massacres_cum','ext_killed','ext_killed_cum',
                 'ext_killed_children_cum','ext_killed_women_cum','ext_injured','ext_injured_cum','ext_civdef_killed_cum',
                 'ext_med_killed_cum','ext_press_killed_cum',],axis = 1)


In [58]:
# Converte a data para datetime
df['report_date'] = pd.to_datetime(df['report_date'])

In [59]:
# Cria coluna: dias desde o início da guerra
inicio_guerra = pd.to_datetime('2023-10-06')
df['dias_desde_inicio'] = (df['report_date'] - inicio_guerra).dt.days

In [60]:
colunas = [
    'killed_cum',
    'injured_cum',
    'med_killed_cum',
    'press_killed_cum',
    'killed_children_cum',
    'killed_women_cum',
    'civdef_killed_cum'

]

# Substitui 0 por NA e faz forward fill em cada coluna
df[colunas] = df[colunas].replace(0, pd.NA).ffill()

/tmp/ipython-input-60-527717218.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[colunas] = df[colunas].replace(0, pd.NA).ffill()


In [61]:
colunas_categorias = [
    'Killed_cum',
    'killed_women_cum',
    'med_killed_cum',
    'press_killed_cum',
    'killed_children_cum',
    'civdef_killed_cum',
    'injured_cum',
]

In [62]:
# Filtra apenas os dados a partir de 7 de outubro de 2023
df = df[df['report_date'] >= inicio_guerra]


In [63]:
df_renomeado = df.rename(columns={
    'killed_cum': 'Geral',
    'killed_women_cum': 'Mulheres',
    'med_killed_cum': 'Médicos',
    'press_killed_cum': 'Imprensa',
    'killed_children_cum': 'Crianças',
    'civdef_killed_cum': 'DefesaCivil'
})

In [64]:
colunas_a_derreter = ['Geral', 'Mulheres', 'Médicos', 'Imprensa', 'Crianças', 'DefesaCivil']
id_vars = [col for col in df_renomeado.columns if col not in colunas_a_derreter]

# Agora sim, faz o melt mantendo tudo
df_long = pd.melt(
    df_renomeado,
    id_vars=id_vars,
    value_vars=colunas_a_derreter,
    var_name='Categoria',
    value_name='Mortos'
)

In [71]:
# Chaves para agrupamento - ajustadas para seu dataset
grupo_chaves = ['report_date', 'report_source', 'report_period', 'dias_desde_inicio']

# Separa a categoria Geral
df_geral = df_long[df_long['Categoria'] == 'Geral'].copy()

# Outras categorias
df_outras = df_long[df_long['Categoria'] != 'Geral'].copy()

# Soma as outras categorias por chave
soma_outras = df_outras.groupby(grupo_chaves)['Mortos'].sum().reset_index()
soma_outras = soma_outras.rename(columns={'Mortos': 'Soma_Outras'})

# Junta com Geral para calcular Homens
df_homens = pd.merge(df_geral, soma_outras, on=grupo_chaves)

# Calcula homens = Geral - soma das outras categorias
df_homens['Mortos'] = df_homens['Mortos'] - df_homens['Soma_Outras']

# Ajusta a categoria para Homens
df_homens['Categoria'] = 'Homens'

# Remove coluna auxiliar
df_homens = df_homens.drop(columns=['Soma_Outras'])

# Junta tudo, excluindo Geral
df_final = pd.concat([df_outras, df_homens], ignore_index=True)

# Ordena se quiser
df_final = df_final.sort_values(by=grupo_chaves + ['Categoria']).reset_index(drop=True)

# Salva ou continua o processamento
df_final.to_csv('dados.csv', index=False)


In [66]:
df_long.to_csv('dados.csv')

In [67]:
print(df_long.columns)

Index(['report_date', 'report_source', 'report_period', 'killed', 'injured_cum', 'injured', 'massacres_cum', 'killed_recovered', 'killed_truce_new', 'killed_succumbed', 'killed_committee', 'dias_desde_inicio', 'Categoria', 'Mortos'], dtype='object')
